In [1]:
import pickle
import random
import time
import os

import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from matplotlib import pyplot as plt
import soundfile as sf
import librosa
import librosa.display

tqdm.pandas()

import paule
import utils
from utils import speak, normalize_cp, inv_normalize_cp, normalize_mel_librosa, inv_normalize_mel_librosa, stereo_to_mono, librosa_melspec, pad_same_to_even_seq_length, RMSELoss, mel_to_sig
from model_zoo import *
from training import pad_batch_online
from matplotlib import cm
DIR = os.getcwd()

/Users/paule/anaconda3/envs/bonn17/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DATA_DIR = "/Volumes/T7/Common_Voice/data/"
RESULT_DICT = "/Volumes/T7/Common_Voice/common_voice_test_planning"
SAVE_DICT = "/Volumes/T7/Common_Voice/common_voice_test_planning/results_acoustic_lr_001_lr_model_00001_10_outer_50_inner_6_batches_2_epochs_only_new"

# Initialize Paule

In [5]:
paule_model = paule.Paule()

# Prepare Test Data

In [ ]:
test_data = pd.read_pickle(DATA_DIR + "common_voice_geco_words_test_subset_slim_prot4.pkl")

## Create Flac Files 

In [ ]:
"""
for i,row in test_data.iterrows():
    sr = 48000
    sig = row.wav_rec
    file_name = row.file_name
    label = row.label
    
    path = RESULT_DICT + label + "/"
    if os.path.isdir(path):
        file = path +  file_name + "_" + label + ".flac"
        if file.is_file():
            j = 1
            while file.is_file(): 
                file = path +  file_name + "_" + label + "%d.flac" % j
                j+=1
        sf.write(path +  file_name + "_" + label + "3.flac" ,sig, sr)
    else:
        os.mkdir(path)
        sf.write(path +  file_name + "_" + label + ".flac" ,sig, sr)
"""

## Random Shuffle Files

In [ ]:
files = []
for dic in os.listdir(RESULT_DICT):
    if (".DS_Store" in dic) or ("results" in dic) : # .DS_Store stores custom attributes of its containing folder in macOS
        continue
    else:
        if not os.path.isdir(os.path.join(SAVE_DICT,dic)):
            os.mkdir(os.path.join(SAVE_DICT,dic))
        path = os.path.join(RESULT_DICT,dic)
        for file in os.listdir(path):
            if (".DS_Store" in file) or ("._" in file):
                continue
            else:
                file = os.path.join(path,file)
                files.append(file)
random.seed(30112021)
random.shuffle(files)

### Omit already planned files

In [ ]:
already_planned_files = []
for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if "planned" in file: 
                    planned_file = "_".join(file.split("_")[:-1]) + ".flac" 
                    already_planned_files.append(planned_file)

unplanned_files = [file for file in files if file.split("/")[-1] not in already_planned_files]

### Load predictive model already used and further learned during planning 

In [ ]:
with open(SAVE_DICT + '/pred_model_50.pkl', 'rb') as pfile:
    model, optimizer = pickle.load(pfile)

In [ ]:
paule_model.pred_model = model
paule_model.pred_optimizer = optimizer

# Start Planning and store results 

In [ ]:
for i,file in enumerate(unplanned_files):
    i += 0
    target_acoustic = file #path + "/" + file
    save_file = SAVE_DICT + "/" + os.path.dirname(file).split("/")[-1] +"/"+os.path.basename(file)[:-5]
    results = paule_model.plan_resynth(learning_rate_planning=0.01, learning_rate_learning=0.0001,
                         target_acoustic=target_acoustic,
                         initialize_from = "acoustic",
                         objective = "acoustic",
                         n_outer=10, n_inner=50,
                         continue_learning = True,
                         add_training_data = False,
                         log_ii = 1,
                         log_semantics=True,
                         n_batches=6, batch_size=8, n_epochs=2,
                         log_gradients = False,
                         plot=True,plot_save_file=save_file, seed=None,
                         verbose=True)

    
    #save_model
    with open(SAVE_DICT + "/pred_model" + "_%d" % (i) + ".pkl", "wb") as pfile:
        m = results[-2]
        o = results[-1]
        pickle.dump((m, o), pfile)

    #save_results
    with open(save_file + '.pkl', 'wb') as f:
        pickle.dump(results[:-2], f)

    # save loss plot
    #plot_save_file = path + "/" + "loss_" + file[:-5] + ".png"
    plot_save_file = SAVE_DICT + "/" + os.path.dirname(file).split("/")[-1] + "/" + "loss_" + os.path.basename(file)[:-5] + ".png"
    prod_loss = results[7]
    planned_loss = results[8]
    planned_mel_loss = results[9] 
    
    fig, ax = plt.subplots(figsize = (15,8), facecolor = "white")
    ax.plot(planned_loss, label = "planned loss", c="C0")
    ax.legend()
    fig.savefig(plot_save_file)
    
    plot_save_file = SAVE_DICT + "/" + os.path.dirname(file).split("/")[-1] + "/" + "mel_loss_" + os.path.basename(file)[:-5] + ".png"
    fig, ax = plt.subplots(figsize = (15,8), facecolor = "white")
    ax.plot(prod_loss, label = "produced mel loss", c = "C1")
    ax.plot(planned_mel_loss, label = "planned mel loss", c="C0")
    ax.legend()
    fig.savefig(plot_save_file)

    # save subloss plot
    plot_save_file = SAVE_DICT + "/" + os.path.dirname(file).split("/")[-1] + "/" + "subloss_" + os.path.basename(file)[:-5] + ".png"
    vel_loss = results[10]
    jerk_loss = results[11]

    fig, ax = plt.subplots(figsize = (15,8), facecolor = "white")
    ax.plot(vel_loss, label = "vel loss", c = "C2")
    ax.plot(jerk_loss, label = "jerk loss", c="C3")
    ax.legend()
    fig.savefig(plot_save_file)

    # save semvec loss plot
    plot_save_file = SAVE_DICT + "/" + os.path.dirname(file).split("/")[-1] + "/" + "semvec_loss_" + os.path.basename(file)[:-5] + ".png"
    pred_semvec_loss = results[12]
    prod_semvec_loss = results[13]

    fig, ax = plt.subplots(figsize = (15,8), facecolor = "white")
    ax.plot(pred_semvec_loss, label = "planned semvec loss", c = "C0")
    ax.plot(prod_semvec_loss, label = "produced semvec loss", c="C1")
    ax.legend()
    fig.savefig(plot_save_file)

    # save inv and planned flac
    sigs = results[18]
    prod_sr = 44100
    sig_inv = sigs[0]
    sf.write(save_file + "_inv.flac" ,sig_inv, prod_sr)

    sig_planned = sigs[-1]
    sf.write(save_file + "_planned.flac" ,sig_planned, prod_sr)

# Plot Model Loss for continued learning 

In [ ]:
model_loss = []
for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if ".pkl" in file: 
                    with open(os.path.join(path,file), 'rb') as f:
                        results = pickle.load(f)
                        model_loss += list((np.asarray(results[21][:-1:2]) + np.asarray(results[21][1::2]))/2)#results[21]

In [ ]:
fig, ax = plt.subplots(figsize = (15,8), facecolor = "white")
ax.plot(model_loss, label = "model loss", c = "C0")
#ax.plot(range(len(model_loss)),np.repeat(np.mean(model_loss),len(model_loss)) ,c = "C1")
ax.legend()